In [1]:
# (C) Copyright IBM Corp. 2019, 2020, 2021, 2022.

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#           http://www.apache.org/licenses/LICENSE-2.0

#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.

In [ ]:
import os
from argparse import ArgumentParser

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from simulai.file import SPFile
from simulai.optimization import Optimizer
from simulai.residuals import SymbolicOperator

In [ ]:
save_path = '.'

In [ ]:
Q = 1_000
N = int(5e4)

In [ ]:
initial_state_test = np.array([1, 1])

In [ ]:
t_intv = [0, 1]
s_intv = np.stack([[-6, -6], [6, 6]], axis=0)

#### The system of equations we aim at solving.

In [ ]:
f_s1 = "D(s1, t) - s2"
f_s2 = "D(s2, t) + b*s2/m + g*sin(s1)/L"

In [ ]:
U_t = np.random.uniform(low=t_intv[0], high=t_intv[1], size=Q)
U_s = np.random.uniform(low=s_intv[0], high=s_intv[1], size=(N, 2))

In [ ]:
np.save("initial_states.npy", U_s)

In [ ]:
branch_input_train = np.tile(U_s[:, None, :], (1, Q, 1)).reshape(N * Q, -1)
trunk_input_train = np.tile(U_t[:, None], (N, 1))

In [ ]:
branch_input_test = np.tile(initial_state_test[None, :], (Q, 1))
trunk_input_test = np.sort(U_t[:, None], axis=0)

In [ ]:
initial_states = U_s

In [ ]:
input_labels = ["t"]
output_labels = ["s1", "s2"]

In [ ]:
n_inputs = len(input_labels)
n_outputs = len(output_labels)

In [ ]:
lambda_1 = 0.0  # Penalty for the L¹ regularization (Lasso)
lambda_2 = 0.0  # Penalty factor for the L² regularization
n_epochs = 300_000  # Maximum number of iterations for ADAM
lr = 1e-3  # Initial learning rate for the ADAM algorithm

In [ ]:
def model():
    
    import numpy as np

    from simulai.models import DeepONet, ImprovedDenseNetwork
    from simulai.regression import SLFNN, ConvexDenseNetwork

    n_latent = 100
    n_inputs_b = 2
    n_inputs_t = 1
    n_outputs = 2

    # Configuration for the fully-connected trunk network
    trunk_config = {
        "layers_units": 7 * [100],  # Hidden layers
        "activations": "tanh",
        "input_size": n_inputs_t,
        "output_size": n_latent * n_outputs,
        "name": "trunk_net",
    }

    # Configuration for the fully-connected branch network
    branch_config = {
        "layers_units": 7 * [100],  # Hidden layers
        "activations": "tanh",
        "input_size": n_inputs_b,
        "output_size": n_latent * n_outputs,
        "name": "branch_net",
    }

    encoder_u_trunk = SLFNN(input_size=n_inputs_t, output_size=100, activation="tanh")
    encoder_v_trunk = SLFNN(input_size=n_inputs_t, output_size=100, activation="tanh")
    encoder_u_branch = SLFNN(input_size=n_inputs_b, output_size=100, activation="tanh")
    encoder_v_branch = SLFNN(input_size=n_inputs_b, output_size=100, activation="tanh")

    # Instantiating and training the surrogate model
    trunk_net_dense = ConvexDenseNetwork(**trunk_config)
    branch_net_dense = ConvexDenseNetwork(**branch_config)

    trunk_net = ImprovedDenseNetwork(
        network=trunk_net_dense, encoder_u=encoder_u_trunk, encoder_v=encoder_v_trunk
    )

    branch_net = ImprovedDenseNetwork(
        network=branch_net_dense, encoder_u=encoder_u_branch, encoder_v=encoder_v_branch
    )

    # It prints a summary of the network features
    trunk_net.summary()
    branch_net.summary()

    pendulum_net = DeepONet(
        trunk_network=trunk_net,
        branch_network=branch_net,
        var_dim=n_outputs,
        devices="gpu",
        model_id="pendulum_net",
    )

    return pendulum_net


In [ ]:
pendulum_net = model()

In [ ]:
residual = SymbolicOperator(
    expressions=[f_s1, f_s2],
    input_vars=input_labels,
    output_vars=output_labels,
    function=pendulum_net,
    inputs_key="input_trunk",
    constants={"b": 0.05, "g": 9.81, "L": 1, "m": 1},
    device="gpu",
    engine="torch",
)

Maximum derivative magnitudes to be used as loss weights

In [ ]:
penalties = [1, 1]
batch_size = 10_000

In [ ]:
optimizer_config = {"lr": lr}

In [ ]:
input_data = {"input_branch": branch_input_train, "input_trunk": trunk_input_train}

In [ ]:
optimizer = Optimizer(
    "adam",
    params=optimizer_config,
    lr_decay_scheduler_params={
        "name": "ExponentialLR",
        "gamma": 0.9,
        "decay_frequency": 5_000,
    },
)

In [ ]:
params = {
    "lambda_1": lambda_1,
    "lambda_2": lambda_2,
    "residual": residual,
    "initial_input": {"input_trunk": np.zeros((N, 1)), "input_branch": initial_states},
    "initial_state": initial_states,
    "weights_residual": [1, 1],
    "weights": penalties,
}

In [ ]:
optimizer.fit(
    op=pendulum_net,
    input_data=input_data,
    n_epochs=n_epochs,
    loss="opirmse",
    params=params,
    device="gpu",
    batch_size=batch_size,
)

Saving model

In [ ]:
print("Saving model.")
saver = SPFile(compact=False)
saver.write(
    save_dir=save_path, name="pendulum_deeponet", model=pendulum_net, template=model
)

In [ ]:
approximated_data = pendulum_net.eval(
    trunk_data=trunk_input_test, branch_data=branch_input_test
)

In [ ]:
for ii in range(n_outputs):
    plt.plot(approximated_data[:, ii], label="Approximated")
    plt.legend()
    plt.savefig(f"pendulum_deeponet_time_int_{ii}.png")
    plt.show()